In [ ]:
using Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

In [ ]:
using RigidBodyDynamics
using RigidBodySim
using MeshCatMechanisms
using StrandbeestRobot
using Rotations: RotMatrix
using StaticArrays: SVector

In [ ]:
mechanism = StrandbeestRobot.mechanism()
gui = GUI(mechanism, URDFVisuals(StrandbeestRobot.urdfpath()));

In [ ]:
#NBSKIP
open(gui);

In [ ]:
state = MechanismState(mechanism)
StrandbeestRobot.solve_initial_configuration!(state)

floatingjoint = findjoint(mechanism, "floating_base")
tf = Transform3D(frame_after(floatingjoint), frame_before(floatingjoint), one(RotMatrix{3}), SVector(0, 0, 0.9))
set_configuration!(state, floatingjoint, tf)

copyto!(gui.visualizer, state)

In [ ]:
# Simulate the robot
control! = let I = velocity_range(state, findjoint(mechanism, "joint_crossbar_crank"))
    function (τ, t, state)
        τ[I] .= -10.0 .* (velocity(state)[I] .- 2.0)
    end
end
dynamics = Dynamics(mechanism, control!)
problem = ODEProblem(dynamics, state, (0, 5.0))
@time solution = solve(problem, Tsit5(), abs_tol=1e-6, dt=1e-6)
setanimation!(gui.visualizer, solution)